# Code generation for recommender workflows

Anton Antonov  
RakuForPrediction at WordPress   
October 2025

----

## Introduction

This notebook demonstrates several different ways to generate Raku code for the package "ML::SparseMatrixRecommender". Both grammar-based interpreters and LLM-based translators are utilized.

----

## Setup

In [108]:
use Data::Reshapers;
use Data::Importers;
use Data::Summarizers;

use DSL::Translators;
use DSL::Examples;
use ML::NLPTemplateEngine;

use Math::SparseMatrix :ALL;
use Math::SparseMatrix::DOK;
use Math::SparseMatrix::Utilities;

use ML::SparseMatrixRecommender;

----

## Ingestion

Ingest a Titanic data CSV file from the Web:

In [109]:
my $url = 'https://raw.githubusercontent.com/antononcube/MathematicaVsR/refs/heads/master/Data/MathematicaVsR-Data-Titanic.csv';
my @dsData = data-import($url, headers => 'auto');
#@dsData .= map({ $_<id> = 'id.' ~ $_<id>; $_ });
#@dsData .= map({ $_<passengerAge> = $_<passengerAge>.Int; $_ });
@dsData.&dimensions

(1309 5)

Show that the data is a list of maps:

In [110]:
deduce-type(@dsData);

Vector(Assoc(Atom((Str)), Atom((Str)), 5), 1309)

---

## SMR

Create a recommender object over Titanic data:

In [111]:
my $smrObj = 
    ML::SparseMatrixRecommender.new
    .create-from-wide-form(@dsData,
        item-column-name => "id",
        tag-types => <passengerSex passengerClass passengerAge passengerSurvival>,
        :add-tag-types-to-column-names,
        tag-value-separator => ":")
    .apply-term-weight-functions("IDF", "None", "Cosine")

ML::SparseMatrixRecommender(:matrix-dimensions((1309, 17)), :density(<4/17>), :tag-types(("passengerClass", "passengerSurvival", "passengerSex", "passengerAge")))

Here is a profile-based recommendation pipeline:

In [112]:
#% html
my @field-names = 'score', 'id', |@dsData.head.keys.grep(* ne 'id').sort;
$smrObj
.recommend-by-profile({"passengerSex:male" => 1, "passengerAge:30" => 20}, 12)
.join-across(@dsData)
.take-value
==> to-html(:@field-names)   

score,id,passengerAge,passengerClass,passengerSex,passengerSurvival
1,1022,30,3rd,male,died
1,1032,30,3rd,male,died
1,1052,30,3rd,male,died
1,1060,30,3rd,male,died
1,1067,30,3rd,male,died
1,1084,30,3rd,male,died
1,1088,30,3rd,male,died
1,1089,30,3rd,male,survived
1,111,30,1st,male,died
1,1111,30,3rd,male,died


In [113]:
$smrObj.take-matrices.keys

(passengerClass passengerSurvival passengerSex passengerAge)

Here is a (history) recommendation pipeline:

In [114]:
#% html
my @field-names = [|<score id>, |$smrObj.take-matrices.keys.sort];
sink $smrObj
.recommend(<1089 1132>, 10, :!remove-history)
.echo-value
.join-across(@dsData, on => 'id')
.echo-value(as => {&to-pretty-table($_, :@field-names)});

[1089 => 8 1132 => 8 1197 => 8 1229 => 8 614 => 8 631 => 8 675 => 8 715 => 8 747 => 8 761 => 8]
+----------+------+--------------+----------------+--------------+-------------------+
|  score   |  id  | passengerAge | passengerClass | passengerSex | passengerSurvival |
+----------+------+--------------+----------------+--------------+-------------------+
| 8.000000 | 1089 |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 1132 |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 1197 |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 1229 |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 614  |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 631  |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 675  |      30      |      3rd       |     male     |      survived     |
| 8.000000 | 715  |      30      |

----

## DSL translation (grammars, ProdGDT)

Here a recommender pipeline specified with natural language commands is translated into R code using the ProdGDT Web service:

In [115]:
'
create from @dsData; 
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st
'
==> dsl-web-translation(to => 'R', format => 'CODE')

SMRMonCreate(data = @dsData) %>%
SMRMonApplyTermWeightFunctions(globalWeightFunction = "IDF", localWeightFunction = "None", normalizerFunction = "Cosine") %>%
SMRMonRecommendByProfile( profile = c("passengerSex:male", "passengerClass:1st"))

Here is similar pipeline is translated with a sub of the package "DSL::Translators":

In [116]:
'
create from @dsData; 
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st;
join across with @dsData on "id";
echo the pipeline value
'
==> ToDSLCode(to => 'Raku', format => 'CODE')
==> {.subst('.', "\n."):g}()

my $obj = ML::SparseMatrixRecommender
.new
.create-from-wide-form(@dsData)
.apply-term-weight-functions(global-weight-func => "IDF", local-weight-func => "None", normalizer-func => "Cosine")
.recommend-by-profile(["passengerSex:male", "passengerClass:1st"])
.join-across(@dsData, on => "id" )
.echo-value()

----

## DSL Translation (LLM examples)

Show known DSL translation examples in "DSL::Examples":

In [117]:
#% html
dsl-examples().map({ $_.key X $_.value.keys }).flat(1).map({ <language workflow> Z=> $_ })».Hash.sort.Array
==> to-dataset()
==> to-html(field-names => <language workflow>)

language,workflow
Python,LSAMon
Python,QRMon
Python,SMRMon
R,LSAMon
R,QRMon
R,SMRMon
Raku,SMRMon
WL,ClCon
WL,LSAMon
WL,QRMon


Define an LLM translation function:

In [118]:
my &llm-pipeline-segment = llm-example-function(dsl-examples()<Raku><SMRMon>);

LLM::Function(-> **@args, *%args { #`(Block|6267625434568) ... }, 'chatgpt')

Here is a recommender pipeline specified with natural language commands:

In [119]:
my $spec = q:to/END/;
new recommender;
create from @dsData; 
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st;
join across with @dsData on "id";
echo the pipeline value;
classify by profile passengerSex:female, and passengerClass:1st on the tag passengerSurvival;
echo value
END

sink my @commands = $spec.lines;

Translate to Raku code:

In [120]:
@commands
.map({ .&llm-pipeline-segment })
.map({ .subst(/:i Output \h* ':'?/, :g).trim })
.join("\n.")

ML::SparseMatrixRecommender.new
.create(@dsData)
.apply-term-weight-functions('IDF', 'None', 'Cosine')
.recommend-by-profile({'passengerSex' => 'male', 'passengerClass' => '1st'})
.join-across(@dsData, on => 'id')
.echo-value()
.classify-by-profile({'passengerSex.female', 'passengerClass.1st'}, 'passengerSurvival')
.echo-value()

Or just calling directly on `$spec`:

In [121]:
&llm-pipeline-segment($spec)

ML::SparseMatrixRecommender.new;
create(@dsData);
apply-term-weight-functions('IDF', 'None', 'Cosine');
recommend-by-profile({'passengerSex' => 'male', 'passengerClass' => '1st'});
join-across(@dsData, on => 'id');
echo-value();
classify-by-profile('passengerSurvival', {'passengerSex' => 'female', 'passengerClass' => '1st'});
echo-value()

----

## NLP Template Engine

Translated "free text" recommender pipeline specification using `concretize` of "ML::NLPTemplateEngine":

In [122]:
'create with dfTitanic; apply the LSI functions IDF, None, Cosine;recommend by profile 1st and male'
==> concretize(lang => "Raku")


my $smrObj = ML::SparseMatrixRecommender.new
.create-from-wide-form(["male"]set, item-column-name='id', :add-tag-types-to-column-names, tag-value-separator=':')
.apply-term-weight-functions('IDF', 'None', 'Cosine')
.recommend-by-profile(["1st"], 12, :!normalize)
.join-across(["male"]set)
.echo-value();

-----

## LLM graph


Making of an LLM graph that:
- Tries all three translation methods above.
- If the DSL grammar-based method does not work then the LLM-based ones are tried.
- The LLM methods are tried in parallel.
- There is a judge that picks which on of the LLM methods produced better result.    

In [177]:
my %rules =
    dsl-grammar => { 
        eval-function => sub ($spec, $lang = 'Raku') { ToDSLCode($spec, to => $lang, format => 'CODE') }
    },

    llm-examples => { 
        llm-function => 
            sub ($spec, $lang = 'Raku') { 
                my &llm-pipeline-segment = llm-example-function(dsl-examples(){$lang}<SMRMon>);
                my @commands = $spec.lines;
                @commands.map({ .&llm-pipeline-segment }).map({ .subst(/:i Output \h* ':'?/, :g).trim }).join("\n.")
            }
    },

    nlp-template-engine => {
        llm-function => sub ($spec, $lang = 'Raku') { concretize($spec, :$lang) }
    },

    judge => sub ($spec, $lang, $dsl-grammar, $llm-examples, $nlp-template-engine) {
            [
                "Choose the generated code that must fully adheres to the spec:\n",
                $spec,
                "\nfrom the following $lang generation results:\n\n",
                "1) DSL-grammar:\n$dsl-grammar\n",
                "2) LLM-examples:\n$llm-examples\n",
                "3) NLP-template-engine:\n$nlp-template-engine\n",
                "and copy it:"
            ].join("\n\n")
    },
    
    report => {
            eval-function => sub ($spec, $lang, $dsl-grammar, $llm-examples, $nlp-template-engine, $judge) {
                [
                    '# Best generated code',
                    "Three $lang code generations were submitted for the spec:",
                    '```text',
                    $spec,
                    '```',
                    'Here are the results:',
                    to-html( ['dsl-grammar', 'llm-examples', 'nlp-template-engine'].map({ [ name => $_, code => ::('$' ~ $_)] })».Hash.Array, field-names => <name code> ).subst("\n", '<br/>'):g,
                    '## Judgement',
                    $judge
                ].join("\n\n")
            }
    }        
;

my $gBestCode = LLM::Graph.new(%rules)

LLM::Graph(size => 5, nodes => dsl-grammar, judge, llm-examples, nlp-template-engine, report)

In [248]:
my $spec = q:to/END/;
make a brand new recommender with the data @dsData;
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st;
join across with @dsData on "id";
echo the pipeline value;
END

$gBestCode.eval(:$spec, lang => 'Raku')

LLM::Graph(size => 5, nodes => dsl-grammar, judge, llm-examples, nlp-template-engine, report)

In [249]:
#% markdown
$gBestCode.nodes<report><result>

# Best generated code

Three Raku code generations were submitted for the spec:

```text

make a brand new recommender with the data @dsData;
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st;
join across with @dsData on "id";
echo the pipeline value;


```

Here are the results:

<table border="1"><thead><tr><th>name</th><th>code</th></tr></thead><tbody><tr><td>dsl-grammar</td><td></td></tr><tr><td>llm-examples</td><td>ML::SparseMatrixRecommender.new(@dsData)<br/>.apply-term-weight-functions(&#39;IDF&#39;, &#39;None&#39;, &#39;Cosine&#39;)<br/>.recommend-by-profile({&#39;passengerSex.male&#39; =&gt; 1, &#39;passengerClass.1st&#39; =&gt; 1})<br/>.join-across(@dsData, on =&gt; &#39;id&#39;)<br/>.echo-value()</td></tr><tr><td>nlp-template-engine</td><td>my $smrObj = ML::SparseMatrixRecommender.new<br/>.create-from-wide-form([&quot;passengerSex:male&quot;, &quot;passengerClass:1st&quot;]set, item-column-name=&#39;id&#39;, :add-tag-types-to-column-names, tag-value-separator=&#39;:&#39;)<br/>.apply-term-weight-functions(&#39;IDF&#39;, &#39;None&#39;, &#39;Cosine&#39;)<br/>.recommend-by-profile([&quot;passengerSex:male&quot;, &quot;passengerClass:1st&quot;], 12, :!normalize)<br/>.join-across([&quot;passengerSex:male&quot;, &quot;passengerClass:1st&quot;]set)<br/>.echo-value();</td></tr></tbody></table>

## Judgement

ML::SparseMatrixRecommender.new(@dsData)
.apply-term-weight-functions('IDF', 'None', 'Cosine')
.recommend-by-profile({'passengerSex.male' => 1, 'passengerClass.1st' => 1})
.join-across(@dsData, on => 'id')
.echo-value()

### Graph

In [174]:
#% html
$gBestCode.dot(engine => 'dot', :9graph-size, node-width => 1.7, node-color => 'grey', edge-color => 'grey', edge-width => 0.4, theme => 'default'):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.0 (20250921.2048)
 -->
<!-- Pages: 1 -->
 
 
 
 
 judge_Cluster 
 
 
<!-- dsl-grammar -->
 
 dsl-grammar 
 
 dsl-grammar 
 
<!-- judge -->
 
 judge 
 
 judge 
 
<!-- dsl-grammar->judge -->
 
 dsl-grammar->judge 
 
 
 
<!-- report -->
 
 report 
 
 report 
 
<!-- dsl-grammar->report -->
 
 dsl-grammar->report 
 
 
 
<!-- judge->report -->
 
 judge->report 
 
 
 
<!-- lang -->
 
 lang 
 
 lang 
 
<!-- lang->dsl-grammar -->
 
 lang->dsl-grammar 
 
 
 
<!-- lang->judge -->
 
 lang->judge 
 
 
 
<!-- llm-examples -->
 
 llm-examples 
 
 llm-examples 
 
<!-- lang->llm-examples -->
 
 lang->llm-examples 
 
 
 
<!-- nlp-template-engine -->
 
 nlp-template-engine 
 
 nlp-template-engine 
 
<!-- lang->nlp-template-engine -->
 
 lang->nlp-template-engine 
 
 
 
<!-- lang->report -->
 
 lang->report 
 
 
 
<!-- llm-examples->judge -->
 
 llm-examples->judge 
 
 
 
<!-- llm-examples->report -->
 
 llm-examples->report 
 
 
 
<!-- nlp-template-engine->judge -->
 
 nlp-template-engine->judge 
 
 
 
<!-- nlp-template-engine->report -->
 
 nlp-template-engine->report 
 
 
 
<!-- spec -->
 
 spec 
 
 spec 
 
<!-- spec->dsl-grammar -->
 
 spec->dsl-grammar 
 
 
 
<!-- spec->judge -->
 
 spec->judge 
 
 
 
<!-- spec->llm-examples -->
 
 spec->llm-examples 
 
 
 
<!-- spec->nlp-template-engine -->
 
 spec->nlp-template-engine 
 
 
 
<!-- spec->report -->
 
 spec->report

---

## Fallback: DSL-grammar to LLM-examples

Instead of having DSL-grammar and LLM computations running in parallel, we can make LLM-graph in which the LLM computations are invoked if the DSL-grammar parsing-and-interpretation fails. Here is such a graph:

In [295]:
my %rules =
    dsl-grammar => { 
        eval-function => sub ($spec, $lang = 'Raku') { 
            my $res = ToDSLCode($spec, to => $lang, format => 'CODE'); 
            my $checkStr = 'my $obj = ML::SparseMatrixRecommender.new';
            return do with $res.match(/ $checkStr /):g { 
                $/.list.elems > 1 ?? $res.subst($checkStr) !! $res 
            }
        }
    },

    llm-examples => { 
        llm-function => 
            sub ($spec, $lang = 'Raku', $split = False) { 
                my &llm-pipeline-segment = llm-example-function(dsl-examples(){$lang}<SMRMon>);
                return do if $split {
                    my @commands = $spec.lines;
                    @commands.map({ .&llm-pipeline-segment }).map({ .subst(/:i Output \h* ':'?/, :g).trim }).join("\n.")
                } else {
                    &llm-pipeline-segment($spec).subst(";\n", "\n."):g
                }
            },
        test-function => sub ($dsl-grammar) { !$dsl-grammar.defined || $dsl-grammar ~~ Str:D && !$dsl-grammar.trim}
    },
    
    code => {
            eval-function => sub ($dsl-grammar, $llm-examples) {
                $dsl-grammar ~~ Str:D && $dsl-grammar.trim ?? $dsl-grammar !! $llm-examples
            }
    }        
;

my $gRobust = LLM::Graph.new(%rules)

LLM::Graph(size => 3, nodes => code, dsl-grammar, llm-examples)

Here the LLM graph run over a spec that can be parsed by DSL-grammar (notice the very short computation time):

In [285]:
my $spec = q:to/END/;
create from @dsData; 
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st;
join across with @dsData on "id";
echo the pipeline value;
END

$gRobust.eval(:$spec, lang => 'Raku')

LLM::Graph(size => 3, nodes => code, dsl-grammar, llm-examples)

Here is the obtained result:

In [286]:
$gRobust.nodes<code><result>

my $obj = ML::SparseMatrixRecommender.new.create-from-wide-form(@dsData).apply-term-weight-functions(global-weight-func => "IDF", local-weight-func => "None", normalizer-func => "Cosine").recommend-by-profile(["passengerSex:male", "passengerClass:1st"]).join-across(@dsData, on => "id" ).echo-value()

Here is the spec cannot be parsed by DSL-grammar interpreter:

In [296]:
my $spec = q:to/END/;
new recommender with @dsData, please; 
apply LSI functions IDF, None, Cosine; 
recommend by profile for passengerSex:male, and passengerClass:1st;
join across with @dsData on "id";
echo the pipeline value;
END

$gRobust.eval(:$spec, lang => 'Raku')

Cannot parse the command; target 'new recommender with @dsData, please' position 0; parsed '', un-parsed 'new recommender with @dsData, please' .


LLM::Graph(size => 3, nodes => code, dsl-grammar, llm-examples)

Nevertheless, we obtain result via LLM-examples:

In [298]:
$gRobust.nodes<code><result>

ML::SparseMatrixRecommender.new(@dsData)
.apply-term-weight-functions('IDF', 'None', 'Cosine')
.recommend-by-profile({'passengerSex.male' => 1, 'passengerClass.1st' => 1})
.join-across(@dsData, on => 'id')
.echo-value();

Here is the corresponding graph plot:

In [299]:
#% html
$gRobust.dot(engine => 'dot', :5graph-size, node-width => 1.7, node-color => 'grey', edge-color => 'grey', edge-width => 0.4, theme => 'default'):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.0.0 (20250921.2048)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- code -->
 
 code 
 
 code 
 
<!-- dsl-grammar -->
 
 dsl-grammar 
 
 dsl-grammar 
 
<!-- dsl-grammar->code -->
 
 dsl-grammar->code 
 
 
 
<!-- llm-examples -->
 
 llm-examples 
 
 llm-examples 
 
<!-- dsl-grammar->llm-examples -->
 
 dsl-grammar->llm-examples 
 
 
 
<!-- lang -->
 
 lang 
 
 lang 
 
<!-- lang->dsl-grammar -->
 
 lang->dsl-grammar 
 
 
 
<!-- lang->llm-examples -->
 
 lang->llm-examples 
 
 
 
<!-- llm-examples->code -->
 
 llm-examples->code 
 
 
 
<!-- spec -->
 
 spec 
 
 spec 
 
<!-- spec->dsl-grammar -->
 
 spec->dsl-grammar 
 
 
 
<!-- spec->llm-examples -->
 
 spec->llm-examples